# Title

# Introduction

provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your report

clearly state the question you tried to answer with your project

identify and describe the dataset that was used to answer the question

## Methods and results

describe in written English the methods you used to perform your analysis from beginning to end that narrates the code the does the analysis.

your report should include code which:

loads data from the original source on the web 

wrangles and cleans the data from it's original (downloaded) format to the format necessary for the planned analysis

performs a summary of the data set that is relevant for exploratory data analysis related to the planned analysis 

creates a visualization of the dataset that is relevant for exploratory data analysis related to the planned analysis

performs the data analysis

creates a visualization of the analysis 

note: all tables and figure should have a figure/table number and a legend

### Methods description

Choosing predictors: 

- After comparing the correlations between the target variable and each predictor, we chose to consider the predictors with at least a correlation > 1. Hence, we chose the following 3 predictors: SCG (corr=0.210), LPR (corr=0.264), PEG (corr=0.488). The other 2 predictors have relatively much smaller correlations (0.075 and 0.061).

We will build predictive models through the K-nearest neighbor classification algorithm, and evaluate to find the accuracy-maximizing classifier. Our methods involve:
- Create a recipe to centre and scale data.
- Cross-validate and tune the classifier and  to find appropriate number of neighbors.
- Create a model followed by workflow to fit the training data.
- Filter out accuracies and plot against the number of neighbours.
- Fit the best model to testing data.
- Other than the line plot of accuracies vs. number of neighbours, we can also create scatterplot of predictor variables and color the points by outcome level. We can also have side-by-side bar plots showing actual and predicted number of knowledge levels. 

## Discussion

summarize what you found

discuss whether this is what you expected to find?

discuss what impact could such findings have?

discuss what future questions could this lead to?

## References

At least 2 citations of literature relevant to the project (format is your choice, just be consistent across the references).
Make sure to cite the source of your data as well.